***In this notebook I'll gathe rsome of the insights and personal learnings from this project.***
Since I am learning the Software development side of Data Science and ML I want to reflect on the lessons learned here. 

A first focus of mine is the testing of Code, since until now I mostly handled it by picking the most difficult samples in my dataset, parsed thenm through the pipelines and even when writing the pipelines thought about "how can I make this stable in case of difficulty/irregularity xyz, this worked fine for research projects with a maximum number of 3 people working on the code (all farmiliar with posible irregularities), but it fails in a larger setup. 



In the Dashboard I want to allow users to pick a Stock of theris choice by using the ISIN or ticker symbol. To communicate with yfinance I need to use the ticker symbol so I wrote a function that turns the ISIN into the ticker symbol. 

The initial one looks like this but has a lot of problems. 

In [ ]:
import yfinance as yf #does not really matter its not supposed to work here

def isin_ticker_to_ticker(ticker_isin:str):
    '''checks if the input is a ticker or Isin and returns the isin as a ticker for easy use with yfinance'''
    
    if len(ticker_isin) == 12:
        ticker = yf.Ticker(f'{ticker_isin}')
        return ticker.ticker
    else:
        return ticker_isin

I do not handle cases where the isin is not correct or the user types in something else. 
I always output the result as the ticker:isin variable even if it is not valid so consecutive functions might fail to retrieve data etc. 
So I need to make the function much more robust.
 <break>

additionally, i need to make sure that only valid isisns and tickers aenter the function. 
Also the validation process can be made in a way that does not call the internet I/O so we do not have to use the network for this
WE also need to have a solution for the case that something invalid is entered

<break>
IT was pointed out to me that the tests should all be done and doable without a connection to hte internet if possible in order to ensure they are speedy and do not fail. So we  should not use yf.Ticker for testing the code we can use a dummy 

*Concepts to learn here*

- make the validation function more robust by adding the option what kind of function is used to test if the input is correct, for testing use one where the solution is more or less "hardcoded" so no network call is done, for production use the general solution that mght require a network check. 
- separate the logic as much as possible, so checking hte input should be different from transforming the input 
- turns out the quest for learning how to test code turned into a deepdive on ISINs, but in this case it is needed to have a reliable check. 
- Note to mayself Keep functions simpler. separate logic even if thoughts go together more often ! 


**Below is a robost implementation of a check for a given isin.**
<break>

Since there is no easy/offline way withut a lookup for a ticker I ll focus on the isin for now, the Ticker part is below that. 

In [ ]:
#Lets think about possible inputs 

# AAPL, real ticker,
# US0378331005, an ISIN or us
# dasdadaa , random things 
# ,.121212   , numbrs and special symbols
# Isins or tickers that look real but are not 

#So mixed upper and lower case letters, a dash symbol somewhere 

In [144]:
# Checking if an ISIN is Valid 
#Im not the first who encountered the problem so lets see that we can do:
#https://www.geeksforgeeks.org/dsa/how-to-validate-isin-using-regular-expressions/
#https://en.wikipedia.org/wiki/International_Securities_Identification_Number
# the wiki artivce describes the process
#the last number is a check digit 
#also, but not on the website, an isin has a checksum we can check additionally. So lets implement the check of the last digit as well
#This is known as luhn algorithm and can easily be found online as well
#https://en.wikipedia.org/wiki/Luhn_algorithm, even with pseudocode 

import re #we can use regular expressions to check 
isin_appl = ' us-037833100-5'
#Note after finishing this section: this was fun but making something robust to human into is weird. Sensor data is much simpler in that regard

def input_case_insensitive(isin_input):
    '''returns a case insensitive version of the user input'''
    return isin_input.upper().strip().replace(' ', '') #transforms to upper, removes spaces in all positions

def remove_dashes(isin_input: str):
    '''removes the dashes from a seemingly valid isin in case there are any '''
    
    newstr = isin_input.replace('-', '')
    return newstr

def isValid_ISIN_Code(isin_input: str):
    
    ##https://www.geeksforgeeks.org/dsa/how-to-validate-isin-using-regular-expressions/

    # Regex to check valid ISIN Code
    regex = "^[A-Z]{0,1}[0-9A-Z]{9}[0-9]{1}$" #original function contained an option to check for dashes, removed in prev. step now

    # Compile the ReGex
    p = re.compile(regex)

    # If the string is empty
    # return false
    if (isin_input == None):
        return False

    # Return if the string
    # matched the ReGex
    if(re.search(p, isin_input)):
        return True
    else:
        return False
    
#Luhns algo
#for conversion of letters need to look at wiki artivle of isin
#I really did not expect this much depths for such a "simple" task
#convert letters to ascii -55 

def check_luhns(isin_input):
    '''the str will have passed the is isin valid function. 
    checks if the isin is correct by employing luhn algorithm
    #https://en.wikipedia.org/wiki/Luhn_algorithm, even with pseudocode '''
    
    #Takes care of the possible characters in the isin
    transformed_isin = ""
    for ch in isin_input:
        if ch.isdigit():
            transformed_isin+= ch
        else:
            transformed_isin += str(ord(ch)-55)
            
    #Now we can apply the actual algorithm
    checksum = 0
    parity = len(transformed_isin) % 2
    
    for idx, digit in enumerate(transformed_isin[:-1]): #iterate through the isin but omit checksum number 
        number = int(digit)
        if (idx+1) % 2 == parity: #not digit, look pseudocode....
            checksum += number
        elif number > 4:
            checksum += 2* number -9
        else:
            checksum += 2* number

    return int(transformed_isin[-1]) == ((10 - (checksum % 10)) % 10)



In [145]:
isValid_ISIN_Code(isin_input= input_case_insensitive(remove_dashes(isin_appl)))

False

In [146]:
check_luhns(input_case_insensitive(remove_dashes(isin_appl)))

True

**What about thee ticker instead of ISIN**

In [ ]:
#tickers max len is 5.
#si if input len is less than 12 , user might have a space in the input  for a ticker an min len of isin is 12
#we go down the ticker check route, which is always a call to the yfinance api for a lookup. for the test I avoid this by hardcoding an example with APPLE
#using a mock class to ckeck. (this was a fun learning)


In [148]:
def check_ticker(user_input: str, check_function = None):
    '''takes a users input that resembles a ticker and checks if it is valid,
    has an input for the check function to avoid online lookup during testing,
    to check, see if it has a price during market hours'''
    
    try:
        if check_function(user_input).info['regularMarketPrice'] is not None:
            return True
        else: 
            return False
    except Exception:
        return False
    
check_ticker('asdPL', check_function= yf.Ticker)

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ASDPL"}}}


False

In [155]:
check_ticker(input_case_insensitive('aapl'), check_function= yf.Ticker)

True

In [157]:
#For testing purposes we can use a fake class similar to yf.Ticker

class FakeTicker:
    def __init__(self, ticker):
        self.info = {'regularMarketPrice': 1} if ticker == 'AAPL' else {}

In [ ]:
check_ticker(input_case_insensitive('aapl'), check_function= FakeTicker)


True

In [160]:
check_ticker(input_case_insensitive('AAPL'), check_function= FakeTicker)

True

With these tests we now have a lot more code than the ~ 6 lines initially. But now it is a robust system. 
The Input of a user is cleaned, standardized and checked.
- For the ISIN we check for dashes some people use, check the format using a regular expression and use Luhns algorithm to test if it is valid
- For the ticker we do not have a way other than a lookup. But in order to allow testing without calling the yfinance library we have build a simple mock class next to the check of the ticker.